## MPCA implementation w/ Tensorly
---
### The MPCA objective

A set of tensor objects $ \{X_{1},X_{2},\ldots,X_{M}\} $ is availalble for training, where each sample $ X_{i}\in\mathbb{R}^{I_{1}xI_{2}x\cdots xI_{N}},i=1,2,\ldots,M $. The MPCA objective is to project each sample to $ \mathbb{R}^{P_{1}xP_{2}x\cdots xP_{N}} $ with $ P_{i}<I_{i},i=1,\ldots,N $ such that the projected samples hold as much variation (from the original data) as possible.

In order to measure variation, we will employ the **total scatter tensor**:
$$ Y_{T}=\sum_{m=1}^{M}||Y_{m}-\overline{Y}||_{F}^{2} $$

, where $ \overline{Y} $ denotes the **mean projected tensor** ($ \overline{Y}=\frac{1}{M}\sum_{m=1}^{M}y_{m} $). Therefore, the objective of MPCA is to determine the projection matrices $ U^{(1)},U^{(2)},\ldots,U^{(N)} $ such that the total scatter is maximized:

$$ U^{(1)},U^{(2)},\ldots,U^{(N)}=\arg\max_{U^{(1)},U^{(2)},\ldots,U^{(N)}}\sum_{m=1}^{M}||Y_{m}-\overline{Y}||_{F}^{2} $$

### Algorithm

**Input:** A set of tensor objects $ \{X_{1},X_{2},\ldots,X_{M}\} $, $ X_{i}\in\mathbb{R}^{I_{1}xI_{2}x\cdots xI_{N}},i=1,2,\ldots,M $

**Output:** $ N $ projection matrices $ U^{(1)},U^{(2)},\ldots,U^{(N)} $ that project the input tensor objects to lower dimensions while maximizing the total scatter.

    1. Center the data
    2. Initialize Projection matrices (Full Projection Truncation)
    3. MPCA Loop
        for k=1:K
            for n=1:N
                3a. Caclulate the n-mode partial projection # Eq. 6.31 [1] 
                3b. Calculate the n-mode total scatter # Eq. 6.32 [1]
                3c .Set U(n) to be the Pn eigenvectors coresponding to the largest Pn eigenvalues of of the n-mode total scatter

### Scatter Maximization - Reconstruction error minimization

One can easily prove that **maximizing the scatter** in each mode is actually the same as **minimizing the reconstruction error**. We will follow the **Alternating Partial Projection** logic, fixing all projection matrices but one. The second objective can be formulated as: 

$$ \min_{U^{(n)}}||X-Y\times_{1}U^{(1)}\times_{2}U^{(2)}\cdots\times_{N}U^{(N)}||_{F}^{2} $$
where for each projection matric $ U^{(n)}\in R^{I_{n}\times P_{n}},U^{(n)^{T}}U^{(n)}=I_{P_{n}} $ is true. Further:
$$ \min_{U^{(n)}}\sum_{m=1}^{M}||X_{m(n)}-U^{(n)}Y_{m(n)}U_{Φ(n)}^{T}||_{F}^{2}$$
$$ =\min_{U^{(n)}}\sum_{m=1}^{M}||X_{m(n)}-U^{(n)}U^{(n)^{T}}X_{m(n)}U_{Φ(n)}U_{Φ(n)}^{T}||_{F}^{2} $$

The Frobenius norm is **unitary invariant**. This means that the value of the norm is not affected when applying orthogonal transformations (*$ U_{Φ(n)} $ is such transformation as the kronecker product of orthogonal matrices*). Therefore:

$$ \min_{U^{(n)}}\sum_{m=1}^{M}||X_{m(n)}U_{Φ(n)}-U^{(n)}U^{(n)^{T}}X_{m(n)}U_{Φ(n)}{U_{Φ(n)}^{T}U_{Φ(n)}}||_{F}^{2} $$
$$ =\min_{U^{(n)}}\sum_{m=1}^{M}||\hat{Y}_{m(n)}^{(n)}-U^{(n)}U^{(n)^{T}}\hat{Y}_{m(n)}^{(n)}||_{F}^{2} $$

where $ U_{Φ(n)}^{T}U_{Φ(n)}=I_{I_{2}I_{3}...I_{N}} $ and $ \hat{Y}_{m(n)}^{(n)} $ denotes the partial projection to all modes except the n-th. The last equation, in a sense, is the **PCA objective for the n-th mode of the data**, and the solution is given straight from the SVD of $ \hat{Y}_{m(n)}^{(n)} $, by setting $ U^{(n)} $ equal to the $P_{n}$ singular vectors corresponding the the $P_{n}$ most siginificant singular values [3].

All that remains is proving the first sentance of this section. *Is maximizing the scatter the same as minimizing the reconstruction error?*

$$  \min_{U^{(n)}}||X-Y\times_{1}U^{(1)}\times_{2}U^{(2)}\cdots\times_{N}U^{(N)}||_{F}^{2} $$

$$ =\min_{U^{(n)}}\sum_{m=1}^{M}||X_{m(n)}-U^{(n)}Y_{m(n)}U_{Φ(n)}^{T}||_{F}^{2}$$

$$ =\min_{U^{(n)}}\sum_{m=1}^{M}tr((X_{m(n)}-U^{(n)}Y_{m(n)}U_{Φ(n)}^{T})(X_{m(n)}-U^{(n)}Y_{m(n)}U_{Φ(n)}^{T})^{T}) $$

$$ =\min_{U^{(n)}}\sum_{m=1}^{M}tr((X_{m(n)}-U^{(n)}Y_{m(n)}U_{Φ(n)})(X_{m(n)}^{T}-U_{Φ(n)}Y_{m(1)}^{T}U^{(n)^{T}})) $$

$$ =\min_{U^{(n)}}\sum_{m=1}^{M}||X_{m(n)}||^{2}-tr(X_{m(n)}U_{Φ(n)}Y_{m(n)}^{T}U^{(1)^{T}})-tr(U^{(1)}Y_{m(n)}U_{Φ(n)}^{T}X_{m(n)}^{T})+tr(U^{(1)}Y_{m(1)}{U_{Φ(1)}^{T}U_{Φ(1)}}Y_{m(1)}^{T}) $$

By leveraging $ tr(ABCD)=tr(DABC) $ we get:

$$ \min_{U^{(n)}}\sum_{m=1}^{M}||X_{m(n)}||_{F}^{2}-||Y_{m(n)}||_{F}^{2} $$

Because $ ||X_{m(1)}||^{2} $ is determined by the data samples, this can be transformed into the following maximization problem:

$$ \max_{U^{(n)}}\sum_{m=1}^{M}||Y_{m(n)}||_{F}^{2} $$

and provided that our data is centered we reach:

$$ \max_{U^{(n)}}\sum_{m=1}^{M}||Y_{m(n)}-\overline{Y}_{(n)}||_{F}^{2} $$

which is the the **scatter maximization objective**. Therefore, the two objective are interchangable and refer to the same process. This is challenged experimentally in *Example 2*.

---
#### References
[1] Multilinear Subspace Learning: Dimensionality Reduction of Multidimensional Data, Haiping Lu, K. N. Plataniotis, and A. N. Venetsanopoulos, Chapman & Hall/CRC Press Machine Learning and Pattern Recognition Series, Taylor and Francis, ISBN: 978-1-4398572-4-3, 2013.

[2] Haiping Lu, K.N. Plataniotis, and A.N. Venetsanopoulos, "MPCA: Multilinear Principal Component Analysis of Tensor Objects", IEEE Transactions on Neural Networks, Vol. 19, No. 1, Page: 18-39, January 2008.

[3] The Elements of Statistical Learning (2nd edition), Hastie, Tibshirani and Friedman, Springer-Verlag, 2009.

In [1]:
# Import necessary libraries

import tensorly as tl
from scipy.io import loadmat
from scipy.linalg import svd
import numpy as np


In [2]:
# Import dataset

USF17GAL = tl.tensor(loadmat('USF17Gal.mat')['fea3D'],dtype=np.double)
print(USF17GAL.shape)


(32, 22, 10, 731)


In [3]:
class myMPCA():
    '''
    An object-oriented implementation of MPCA (Multilinear Principle Component Analysis).
    
    Methods
    -------
    __init__(): Initialzies the myMPCA object and handles error-checking on parameters the user has entered.
    fit(): Performs MPCA with given arguments on samples.
    transform(): Projects and returns the data using the computed projection matrices from fit().
    
    Attributes
    ----------
    self.maxK: No of iterations (user-defined)
    self.Q: Variance percentage to keep (user defined)
    self.objective: Objective to solve {error_min} or {scatter_max}
    self.sample_modes: # of modes of given dataset
    self.sample_dims: # of modes of a sample
    self.samples_no: # of samples
    self.Ps: The dimensions of the projected samples (list)
    self.samples_mean: Mean tensor of input
    
    '''
    
    def __init__(self,Q,maxK,objective):
        '''
        Q: Percentage of variation to keep
        maxK: Maximum number of iterations
        objective: Use error minization {error_min} or scatter maximation {scatter_max}. These have the very same results as also shown by example 2.
        '''
        
        # Error handling on input
            
        try:
            
            if int(maxK) <= 0:
                
                raise Exception('Max iters (k) has to be a positive integer.')
                
            self.maxK = maxK
                
        except:

            raise Exception('Max iters (k) has to be a positive integer.')

        try:
            
            if float(Q) <= 0:
            
                raise Exception('Q has to be positive.')
                
            self.Q = Q
                
        except:

            raise Exception('Q has to be a positive integer.')
            
        if objective == 'error_min' or objective == 'scatter_max':
            
            self.objective = objective
            
        else:
            
            raise Exception('Not valid objective to use. Please use either {error_min} or {scatter_max}')
            
    def fit(self,samples):
        '''
        Performs MPCA with given arguments.
        samples: Input tensor of (N+1) dimensions (+1: Group input in one larger tensor)
        '''
        
        # Gather and store basic info on the train data
    
        self.sample_modes = len(samples.shape)
    
        self.sample_dims = len(samples.shape) - 1

        self.samples_no = samples.shape[-1]
        
        self.Ps = []
        
        ########################################################
        # 1. Center the data
        ########################################################
        
        # Find mean tensor
        
        self.samples_mean = tl.mean(samples,axis=self.sample_dims)
        
        # Center the data

        for i in range(self.samples_no):

            samples[...,i] = np.subtract(samples[...,i],self.samples_mean)
     
        ########################################################
        # 2. Initialize Projecton Matrices
        ########################################################
        
        # Initializing projection matrices according to Full Projection Truncation...

        self.projection_matrices = []
        self.corresponding_eigenvals = []

        for n in range(self.sample_dims):

            # Phi (n-mode total scatter matrix has dimensions In x In)

            In = samples.shape[n]

            phi = np.zeros((In,In))

            for m in range(self.samples_no):

                # Store the m-th sample in Xm
                
                Xm = samples[...,m]

                # N-mode matricization of Xm

                Xm_unfolded = tl.unfold(Xm,mode=n)

                # Add Xm x Xm ^ T to Phi

                phi = phi + Xm_unfolded @ tl.transpose(Xm_unfolded)

            # Solve the eigenvalue problem on Phi

            eigenvals, eigenvecs = np.linalg.eig(phi)

            # eig doesnt not necessarily return sorted eigenvalues, we need to sort them

            indexOrd = np.argsort(eigenvals)[::-1] 

            eigenvecs_sorted = eigenvecs[:,indexOrd]

            eigenvals_sorted = eigenvals[indexOrd]
            
            # In order to get consistent results, force the first component of
            # each eigenvector to be positive

            for l in range(eigenvecs_sorted.shape[1]):

                if eigenvecs_sorted[0,l] < 0.0:

                    eigenvecs_sorted[:,l] = eigenvecs_sorted[:,l] * (-1)

            # Store eigenvals and eigenvectors (eigenvectors will compose the initial projection matrices)

            self.projection_matrices.append(eigenvecs_sorted)

            self.corresponding_eigenvals.append(eigenvals_sorted)
                
        ########################################################
        # 3. Q-Based method (determine projection dimensions)
        ########################################################
        
        # In short, the Q-Based method finds the ratio of the eigenvalues
        # after truncation of a number of the least important eigenvalues
        # over the sum of all the eigenvalues. This ratio 'explains'
        # the remaining scatter in this node. Moreover, we need this
        # value to be AT LEAST user-defined Q. Therefore, we truncate as many
        # dimensions as possible in respect to this contraint.
        
        sums = [] # Another way to see this ratio is that of the cumulative distribution of the eigenvalues
        
        for n in range(self.sample_dims): # In each mode
            
            # Find the sum of the most significant eigenvalues ...
            
            In = len(self.corresponding_eigenvals[n])
            
            sums.append(np.zeros((In,1)))
            
            sums[n][0] = self.corresponding_eigenvals[n][0]
            
            sum_per_mode = self.corresponding_eigenvals[n][0]
        
            for i in range(1,In):
                
                sums[n][i] = sums[n][i-1] + self.corresponding_eigenvals[n][i]
                
            # ... over the sum of all the eigenvalues for each mode
                
            sums[n] = sums[n] / sum(self.corresponding_eigenvals[n])
            
            # However, we have to keep only the most significant eigenvectors
            # so that the ratio is AT LEAST Q
            
            Ps = np.where(sums[n] >= self.Q/100)
            
            if self.objective == 'scatter_max':
            
                self.projection_matrices[n] = self.projection_matrices[n][:,0:(Ps[0][0]+1)] # Truncating columns
            
            elif self.objective == 'error_min':
                
                self.projection_matrices[n] = np.transpose(self.projection_matrices[n][:,0:(Ps[0][0]+1)]) # Truncating columns                
            
            self.Ps.append(Ps[0][0]+1)
        
        ########################################################
        # 4. MPCA Loop
        ########################################################
        
        if self.objective == 'scatter_max':
    
            for k in range(self.maxK):

                for n in range(self.sample_dims):

                    In = samples.shape[n]

                    phi = np.zeros((In,In))

                    for m in range(self.samples_no): # Eq. 6.32 [1]

                        # Store the m-th sample in Xm

                        Xm = samples[...,m]

                        # (Partially) project Xm using the existing projection matrices

                        projection_modes = [i for i in range(len(Xm.shape))]

                        projection_modes.remove(n) # Without using the current mode!

                        projections2use = [self.projection_matrices[i] for i in projection_modes]

                        Xm = tl.tenalg.multi_mode_dot(Xm,projections2use,modes=projection_modes,transpose=True)

                        # N-mode matricization of Xm

                        Xm_unfolded = tl.unfold(Xm,mode=n)

                        # Add Xm x Xm ^ T to Phi

                        phi = phi + Xm_unfolded @ tl.transpose(Xm_unfolded) 

                    # phi holds now the complete n-mode scatter matrix
                    # Finding the largest eigenvalues and eigenvectors of phi
                    # will lead us to the n-mode projetion matrix.

                    eigenvals, eigenvecs = np.linalg.eig(phi)

                    # eig doesnt not necessarily return sorted eigenvalues, we need to sort them
                    # and keep Pn < In of them in the projection matrix.

                    indexOrd = np.argsort(eigenvals)[::-1]

                    eigenvecs_sorted = eigenvecs[:,indexOrd]

                    eigenvecs_truncated = eigenvecs_sorted[:,0:self.Ps[n]]

                    eigenvals_sorted = eigenvals[indexOrd]

                    eigenvals_truncated = eigenvals_sorted[0:self.Ps[n]]

                    # In order to get consistent results, force the first component of
                    # each eigenvector to be positive

                    for l in range(self.Ps[n]):

                        if eigenvecs_truncated[0,l] < 0.0:

                            eigenvecs_truncated[:,l] = eigenvecs_truncated[:,l] * (-1)

                    # Update projection matrix of this mode

                    self.projection_matrices[n] = eigenvecs_truncated
                    
        elif self.objective == 'error_min':
            
            # Error minimazation

            for k in range(self.maxK):

                for n in range(self.sample_dims):

                    In = samples.shape[n]

                    # Calculate the partial projection

                    projection_modes = [i for i in range(self.sample_modes-1)]

                    projection_modes.remove(n) # Without using the current mode!

                    projections2use = [self.projection_matrices[i] for i in projection_modes]

                    partial_projection = tl.tenalg.multi_mode_dot(samples,projections2use,modes=projection_modes,transpose=False)            

                    partial_projection_unfolded = np.transpose(tl.unfold(partial_projection,mode=n))

                    # Set the U(n) projection matrix equal to Pn first right eigenvectors of Xm_unfolded

                    U, s, Vh = svd(partial_projection_unfolded,full_matrices=False)

                    right_singular_vecs_truncated = Vh[0:self.Ps[n],:]

                    # In order to get consistent results, force the first component of
                    # each eigenvector to be positive

                    for l in range(self.Ps[n]):

                        if right_singular_vecs_truncated[l,0] < 0.0:

                            right_singular_vecs_truncated[l,:] = right_singular_vecs_truncated[l,:] * (-1)                    

                    # Update projection matrix of this mode

                    self.projection_matrices[n] = right_singular_vecs_truncated


    def transform(self,samples):
        '''
        Projects and returns the data using the computed projection 
        matrices from fit().
        '''
        
        if self.objective == 'scatter_max':
        
            projected_data = tl.tenalg.multi_mode_dot(samples,self.projection_matrices,transpose=True)
        
        elif self.objective == 'error_min':
            
            projected_data = tl.tenalg.multi_mode_dot(samples,self.projection_matrices,transpose=False)            
        
        return projected_data


---
## Examples

In [4]:
# Example 1

# Create a myMPCA object and fit the dataset

mpca = myMPCA(97,10,'scatter_max')
mpca.fit(USF17GAL)

# Print projection matrix sizes

for n in range(len(mpca.projection_matrices)):
    print(f'{n}-mode projection matrix shape: {mpca.projection_matrices[n].shape}')
    
# Print initial tensor size and projected tensor size

print(f'Initial tensor size (w/ samples): {USF17GAL.shape}');

projected_data = mpca.transform(USF17GAL)

print(f'Projected tensor size (w/ samples): {projected_data.shape}');


0-mode projection matrix shape: (32, 26)
1-mode projection matrix shape: (22, 15)
2-mode projection matrix shape: (10, 10)
Initial tensor size (w/ samples): (32, 22, 10, 731)
Projected tensor size (w/ samples): (26, 15, 10, 731)


In [5]:
# Example 2

# Create 2 MPCA object for the two objectives and fit the dataset

mpca_error_min = myMPCA(98,10,'error_min')
mpca_scatter_max = myMPCA(98,10,'scatter_max')

mpca_error_min.fit(USF17GAL)
mpca_scatter_max.fit(USF17GAL)

# Project the data

projected_data_error_min = mpca_error_min.transform(USF17GAL)
projected_data_scatter_max = mpca_scatter_max.transform(USF17GAL)

# and print the norm of the distance between the results of each objective

print(f'||T_error_min - T_scatter_max||= {tl.norm(projected_data_error_min-projected_data_scatter_max)}\n')

# along with the norm of the distance between projection matrices

for n in range(len(mpca_error_min.projection_matrices)):
    print(f'{n}-mode projection matrix : {tl.norm(np.transpose(mpca_error_min.projection_matrices[n])-mpca_scatter_max.projection_matrices[n])}')

print('\n')
    
# Finally, print the mode-3 projection matrix of each approach

print('mode-3 projection matrix of error_min approach\n----------------------------------------------')
print(mpca_scatter_max.projection_matrices[2])

print('\n')

print('mode-3 projection matrix of scatter_max approach\n------------------------------------------------')
print(np.transpose(mpca_error_min.projection_matrices[2]))


||T_error_min - T_scatter_max||= 2.142162928584675e-11

0-mode projection matrix : 1.684525025500799e-13
1-mode projection matrix : 1.585987509423577e-13
2-mode projection matrix : 7.057462914935218e-14


mode-3 projection matrix of error_min approach
----------------------------------------------
[[ 0.27384955  0.0174603   0.4914432   0.45367813  0.34185165  0.53956814
   0.19236548  0.17076755  0.03812933  0.04177432]
 [ 0.34598142  0.26084022  0.43993326  0.25336088 -0.07724821 -0.47558074
  -0.3543072  -0.40272969 -0.11536558 -0.14610915]
 [ 0.37110676  0.35589344  0.18531734 -0.26713846 -0.36380175 -0.19323186
   0.22434981  0.50023961  0.2204387   0.33325392]
 [ 0.38696814  0.31480832 -0.14028795 -0.41717297 -0.07905606  0.38233954
   0.2130531  -0.25294391 -0.29465835 -0.45695538]
 [ 0.38619683  0.15722878 -0.4016527  -0.08956554  0.38164136  0.14983162
  -0.38485447 -0.22021594  0.26968532  0.46834432]
 [ 0.36359073 -0.07245699 -0.44128997  0.31590269  0.24795888 -0.31897545
  